## Chatbot project with Flask and Streamlit deployment

In this project we will build a very simple chatbot that will predict a tag/class from which the user input comes and from a list of possible responses randomly choose one to output. 

Our data is stored in the *intents* json file which we will load and preprocess before buidling a model.

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

In [36]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [37]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

The first preprocessing step will be to tokenize every sentence in patterns for each tag. This will basically means that each sentence will be split into words. Words from all the senteces will be stored together in the *words* list which will be used as a vocabulary. We also define a *documents* list which will store every pair of tokenized sentence and corresponding class as a tuple. Finally the *classes* list will contain all unique tags/classes.

In [38]:
words = []
classes = []
documents = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [39]:
classes

['greeting',
 'goodbye',
 'thanks',
 'options',
 'adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'pharmacy_search',
 'hospital_search']

In [40]:
words[:10]

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey']

In [41]:
documents[:10]

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye')]

Now we can *lemmatize* our *words* list ignoring some of the characters and also choose only the unique words to build a final vocabulary.

In [42]:
ignore_letters = ['!', '?', ',', '.']
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

In [43]:
words[:10]

["'s",
 'a',
 'adverse',
 'all',
 'anyone',
 'are',
 'awesome',
 'be',
 'behavior',
 'blood']

We can now save *words* and *classes* lists as pickle files. They will be needed later on when we will build our chatbot.
To encode our patterns for training we can *lemmatize* each pattern and check at which postions words from our pattern appear in the vocabulary. At these positions we enter 1 and otherwise 0. This will encode every pattern as a sparse vector with many zeros. The target will be a class to which every pattern belongs encoded as an integer using a *classes_dict* which maps tags to indices.

In [44]:
classes_dict = {classes[i]: i for i in range(len(classes))}

In [45]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [46]:
training = []
for doc in documents:
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    bag = [1 if word in word_patterns else 0 for word in words]
    target = classes_dict[doc[1]]
    training.append([bag, target])

Now lets shuffle our training list and split for features and targets.

In [47]:
random.shuffle(training)
X = [pattern[0] for pattern in training]
y = [pattern[1] for pattern in training]

Our model will be a simple Sequential network with a *softmax* activation function at the last *Dense* layer. It will output probabilities for every tag for each pattern.

In [48]:
model = keras.models.Sequential([
    keras.layers.Dense(128, input_shape=[len(X[0])], activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(classes), activation='softmax')
])

In [49]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               11264     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 9)                 585       
Total params: 20,105
Trainable params: 20,105
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, epochs=400, batch_size=4, verbose=1)

Epoch 1/400
12/12 [==============================] - 0s 727us/step - loss: 2.2862 - accuracy: 0.0410
Epoch 2/400
12/12 [==============================] - 0s 815us/step - loss: 2.1074 - accuracy: 0.2972
Epoch 3/400
12/12 [==============================] - 0s 816us/step - loss: 2.1979 - accuracy: 0.2069
Epoch 4/400
12/12 [==============================] - 0s 725us/step - loss: 2.1140 - accuracy: 0.1282
Epoch 5/400
12/12 [==============================] - 0s 727us/step - loss: 2.0401 - accuracy: 0.3327
Epoch 6/400
12/12 [==============================] - 0s 730us/step - loss: 2.0815 - accuracy: 0.2046
Epoch 7/400
12/12 [==============================] - 0s 727us/step - loss: 1.9859 - accuracy: 0.2738
Epoch 8/400
12/12 [==============================] - 0s 727us/step - loss: 1.8469 - accuracy: 0.4551
Epoch 9/400
12/12 [==============================] - 0s 636us/step - loss: 1.9432 - accuracy: 0.4217
Epoch 10/400
12/12 [==============================] - 0s 727us/step - loss: 1.8936 - accura

12/12 [==============================] - 0s 816us/step - loss: 0.0989 - accuracy: 1.0000
Epoch 82/400
12/12 [==============================] - 0s 818us/step - loss: 0.0877 - accuracy: 0.9787
Epoch 83/400
12/12 [==============================] - 0s 818us/step - loss: 0.0591 - accuracy: 0.9826
Epoch 84/400
12/12 [==============================] - 0s 799us/step - loss: 0.0566 - accuracy: 1.0000
Epoch 85/400
12/12 [==============================] - 0s 727us/step - loss: 0.1250 - accuracy: 1.0000
Epoch 86/400
12/12 [==============================] - 0s 818us/step - loss: 0.0474 - accuracy: 1.0000
Epoch 87/400
12/12 [==============================] - 0s 818us/step - loss: 0.0887 - accuracy: 0.9675
Epoch 88/400
12/12 [==============================] - 0s 817us/step - loss: 0.0493 - accuracy: 1.0000
Epoch 89/400
12/12 [==============================] - 0s 727us/step - loss: 0.0275 - accuracy: 0.9967
Epoch 90/400
12/12 [==============================] - 0s 818us/step - loss: 0.0559 - accuracy: 

12/12 [==============================] - 0s 727us/step - loss: 0.0741 - accuracy: 0.9296
Epoch 161/400
12/12 [==============================] - 0s 818us/step - loss: 0.0520 - accuracy: 0.9826
Epoch 162/400
12/12 [==============================] - 0s 818us/step - loss: 0.1244 - accuracy: 0.9579
Epoch 163/400
12/12 [==============================] - 0s 818us/step - loss: 0.0126 - accuracy: 1.0000
Epoch 164/400
12/12 [==============================] - 0s 818us/step - loss: 0.0123 - accuracy: 1.0000
Epoch 165/400
12/12 [==============================] - 0s 815us/step - loss: 0.0151 - accuracy: 1.0000
Epoch 166/400
12/12 [==============================] - 0s 818us/step - loss: 0.0189 - accuracy: 1.0000
Epoch 167/400
12/12 [==============================] - 0s 820us/step - loss: 0.0051 - accuracy: 1.0000
Epoch 168/400
12/12 [==============================] - 0s 818us/step - loss: 0.0343 - accuracy: 0.9675
Epoch 169/400
12/12 [==============================] - 0s 815us/step - loss: 0.0520 - a

12/12 [==============================] - 0s 818us/step - loss: 0.0058 - accuracy: 1.0000
Epoch 240/400
12/12 [==============================] - 0s 818us/step - loss: 0.0020 - accuracy: 1.0000
Epoch 241/400
12/12 [==============================] - 0s 818us/step - loss: 0.0011 - accuracy: 1.0000
Epoch 242/400
12/12 [==============================] - 0s 727us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 243/400
12/12 [==============================] - 0s 818us/step - loss: 0.0281 - accuracy: 1.0000
Epoch 244/400
12/12 [==============================] - 0s 727us/step - loss: 0.0297 - accuracy: 1.0000
Epoch 245/400
12/12 [==============================] - 0s 727us/step - loss: 0.0091 - accuracy: 1.0000
Epoch 246/400
12/12 [==============================] - 0s 727us/step - loss: 0.0038 - accuracy: 1.0000
Epoch 247/400
12/12 [==============================] - 0s 816us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 248/400
12/12 [==============================] - 0s 818us/step - loss: 0.0013 - a

12/12 [==============================] - 0s 727us/step - loss: 0.0043 - accuracy: 1.0000
Epoch 319/400
12/12 [==============================] - 0s 818us/step - loss: 6.1938e-04 - accuracy: 1.0000
Epoch 320/400
12/12 [==============================] - 0s 727us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 321/400
12/12 [==============================] - 0s 818us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 322/400
12/12 [==============================] - 0s 818us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 323/400
12/12 [==============================] - 0s 727us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 324/400
12/12 [==============================] - 0s 818us/step - loss: 0.0037 - accuracy: 1.0000
Epoch 325/400
12/12 [==============================] - 0s 727us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 326/400
12/12 [==============================] - 0s 815us/step - loss: 0.0068 - accuracy: 1.0000
Epoch 327/400
12/12 [==============================] - 0s 729us/step - loss: 0.0015

12/12 [==============================] - 0s 817us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 398/400
12/12 [==============================] - 0s 727us/step - loss: 0.0653 - accuracy: 0.9387
Epoch 399/400
12/12 [==============================] - 0s 907us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 400/400
12/12 [==============================] - 0s 818us/step - loss: 2.7208e-04 - accuracy: 1.0000


Now we can save our model and use it later to build the chatbot.

In [51]:
model.save('chatbot_model.h5', model)

Before moving on to the app we can also look at the other way of training our model. Instead of creating a vocabulary and encoding every pattern based on this vocabulary, we can use *tf-idf* vectorizer transformer from sklearn which will convert each word in every pattern to its relative frequency in the entire corpus.

In [52]:
classes = []
documents = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        documents.append((pattern, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [53]:
random.shuffle(documents)
X = [pattern[0] for pattern in documents]
y = [pattern[1] for pattern in documents]

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

In [55]:
print(vectorizer.get_feature_names())

['adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'details', 'do', 'dont', 'drug', 'drugs', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacies', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'results', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [56]:
y = [classes_dict[tag] for tag in y]

In [57]:
model = keras.models.Sequential([
    keras.layers.Dense(128, input_shape=[len(vectorizer.get_feature_names())], activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(classes), activation='softmax')
])

In [58]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X.todense().tolist(), y, epochs=400, batch_size=4, verbose=1)

Epoch 1/400
12/12 [==============================] - 0s 2ms/step - loss: 2.2143 - accuracy: 0.0795
Epoch 2/400
12/12 [==============================] - 0s 816us/step - loss: 2.1144 - accuracy: 0.3499
Epoch 3/400
12/12 [==============================] - 0s 728us/step - loss: 2.1653 - accuracy: 0.1388
Epoch 4/400
12/12 [==============================] - 0s 727us/step - loss: 2.1277 - accuracy: 0.1819
Epoch 5/400
12/12 [==============================] - 0s 727us/step - loss: 2.0473 - accuracy: 0.4287
Epoch 6/400
12/12 [==============================] - 0s 727us/step - loss: 2.0753 - accuracy: 0.2805
Epoch 7/400
12/12 [==============================] - 0s 797us/step - loss: 2.0831 - accuracy: 0.3249
Epoch 8/400
12/12 [==============================] - 0s 727us/step - loss: 2.0271 - accuracy: 0.4296
Epoch 9/400
12/12 [==============================] - 0s 708us/step - loss: 1.9699 - accuracy: 0.4253
Epoch 10/400
12/12 [==============================] - 0s 727us/step - loss: 1.9103 - accuracy

12/12 [==============================] - 0s 727us/step - loss: 0.1545 - accuracy: 0.9387
Epoch 82/400
12/12 [==============================] - 0s 727us/step - loss: 0.0260 - accuracy: 1.0000
Epoch 83/400
12/12 [==============================] - 0s 818us/step - loss: 0.0730 - accuracy: 1.0000
Epoch 84/400
12/12 [==============================] - 0s 727us/step - loss: 0.0249 - accuracy: 1.0000
Epoch 85/400
12/12 [==============================] - 0s 818us/step - loss: 0.0484 - accuracy: 1.0000
Epoch 86/400
12/12 [==============================] - 0s 818us/step - loss: 0.0603 - accuracy: 1.0000
Epoch 87/400
12/12 [==============================] - 0s 727us/step - loss: 0.0865 - accuracy: 0.9885
Epoch 88/400
12/12 [==============================] - 0s 727us/step - loss: 0.0918 - accuracy: 1.0000
Epoch 89/400
12/12 [==============================] - 0s 727us/step - loss: 0.1380 - accuracy: 0.9317
Epoch 90/400
12/12 [==============================] - 0s 818us/step - loss: 0.0302 - accuracy: 

12/12 [==============================] - 0s 727us/step - loss: 0.0801 - accuracy: 0.9579
Epoch 161/400
12/12 [==============================] - 0s 727us/step - loss: 0.0219 - accuracy: 1.0000
Epoch 162/400
12/12 [==============================] - 0s 727us/step - loss: 0.0228 - accuracy: 1.0000
Epoch 163/400
12/12 [==============================] - 0s 818us/step - loss: 0.0537 - accuracy: 0.9579
Epoch 164/400
12/12 [==============================] - 0s 818us/step - loss: 0.0178 - accuracy: 1.0000
Epoch 165/400
12/12 [==============================] - 0s 818us/step - loss: 0.0092 - accuracy: 1.0000
Epoch 166/400
12/12 [==============================] - 0s 818us/step - loss: 0.0205 - accuracy: 1.0000
Epoch 167/400
12/12 [==============================] - 0s 909us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 168/400
12/12 [==============================] - 0s 818us/step - loss: 0.0104 - accuracy: 1.0000
Epoch 169/400
12/12 [==============================] - 0s 818us/step - loss: 0.0083 - a

12/12 [==============================] - 0s 818us/step - loss: 0.0056 - accuracy: 1.0000
Epoch 240/400
12/12 [==============================] - 0s 727us/step - loss: 0.0604 - accuracy: 1.0000
Epoch 241/400
12/12 [==============================] - 0s 727us/step - loss: 0.0179 - accuracy: 0.9931
Epoch 242/400
12/12 [==============================] - 0s 818us/step - loss: 9.9308e-04 - accuracy: 1.0000
Epoch 243/400
12/12 [==============================] - 0s 727us/step - loss: 0.0070 - accuracy: 1.0000
Epoch 244/400
12/12 [==============================] - 0s 818us/step - loss: 0.0225 - accuracy: 0.9909
Epoch 245/400
12/12 [==============================] - 0s 727us/step - loss: 0.0063 - accuracy: 1.0000
Epoch 246/400
12/12 [==============================] - 0s 727us/step - loss: 0.0049 - accuracy: 1.0000
Epoch 247/400
12/12 [==============================] - 0s 818us/step - loss: 0.0029 - accuracy: 1.0000
Epoch 248/400
12/12 [==============================] - 0s 820us/step - loss: 0.0039

12/12 [==============================] - 0s 727us/step - loss: 0.0031 - accuracy: 1.0000
Epoch 319/400
12/12 [==============================] - 0s 818us/step - loss: 0.0037 - accuracy: 1.0000
Epoch 320/400
12/12 [==============================] - 0s 727us/step - loss: 0.0042 - accuracy: 1.0000
Epoch 321/400
12/12 [==============================] - 0s 818us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 322/400
12/12 [==============================] - 0s 727us/step - loss: 0.0029 - accuracy: 1.0000
Epoch 323/400
12/12 [==============================] - 0s 818us/step - loss: 8.0528e-04 - accuracy: 1.0000
Epoch 324/400
12/12 [==============================] - 0s 727us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 325/400
12/12 [==============================] - 0s 818us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 326/400
12/12 [==============================] - 0s 775us/step - loss: 5.7630e-04 - accuracy: 1.0000
Epoch 327/400
12/12 [==============================] - 0s 727us/step - loss: 0.

Epoch 397/400
12/12 [==============================] - 0s 727us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 398/400
12/12 [==============================] - 0s 818us/step - loss: 5.4726e-04 - accuracy: 1.0000
Epoch 399/400
12/12 [==============================] - 0s 637us/step - loss: 9.8703e-04 - accuracy: 1.0000
Epoch 400/400
12/12 [==============================] - 0s 818us/step - loss: 0.0016 - accuracy: 1.0000


As we could see, both models have very similar performance.